# 1.2 Automate LinkedIn demographic data extraction from XLS files

In the previous notebook, I finished automating the extraction of metric data from monthly reports curated from LinkedIn and stored in XLS files. In this one, I want to use one of the XLS files (I believe `company1_visitors.xls`) to automate demographic data extraction and complete the LinkedIn report.

In [2]:
import os
import pandas as pd

ROOT_DIR = os.path.dirname(os.path.abspath("../../setup.py"))
DATA_DIR = os.path.join(ROOT_DIR, "data/raw/linkedin")

In [3]:
file = f"{DATA_DIR}/jotovent-2020-11_visitors.xls"
assert os.path.exists(file)

AssertionError: 

## Demographic Data Tables

Each LinkedIn report wants demographic data from three areas: `Location`, `Industry`, and `Job Function`. All of these tables will require calling out to a database that contains E<->J translations to get the Japanese versions of the English words returned from LinkedIn.

### Location Data Tables

Final `Location` data tables can take on one of two forms. The first is country-level:

|    Country    |  国  | Visitors |
|:-------------:|:----:|:--------:|
| United States | 米国 |       61 |
| China         | 中国 |        9 |
| Taiwan        | 台湾 |        2 |
|               |      |          |
|               |      |          |

The second is area-level:

| Country                  |             国             | Visitors |
|--------------------------|:--------------------------:|:--------:|
| Greater Seattle Area     | シアトルエリア             |       52 |
| Greater Chicago Area     | シカゴエリア               |        4 |
| Greater Los Angeles Area | ロサンゼルスエリア         |        3 |
| San Francisco Bay Area   | サンフランシスコベイエリア |        3 |
| Osaka, Japan             | 大阪                       |        3 |

I'm unsure if I want to forego the area-level table and just report country-level data for each company. Regardless, I need to be able to transform the area-level data into country-level data, because LinkedIn provides location data by area.

### Industry Data Tables

`Industry` data is more standardized:

|               Industry              |        産　業        | Visitors |
|:-----------------------------------:|:--------------------:|:--------:|
| Marketing and Advertising           | マーケティング・広告 |       32 |
| Electrical/Electronic Manufacturing | 電気／電子製造       |       17 |
| Consumer Electronics                | 家電                 |        9 |
| Machinery                           | 機械                 |        7 |
| Information Technology and Services | ITサービス           |        6 |

### Job Function Data Tables

Likewise, `Job Function` data is quite straightforward:

|     Job Function     |      職　務      | Visitors |
|:--------------------:|:----------------:|:--------:|
| Business Development | 事業開発         |       31 |
| Sales                | 販売             |       10 |
| Engineering          | エンジニアリング |        8 |
| Marketing            | マーケティング   |        5 |
| Administrative       | 行政             |        3 |

I'm noticing that in the Google Sheets for these data tables, the cells that contain each value are not aligned across sheets (i.e. Industry index is in column K on one sheet, column L on another). So I'll have to work manually with inputting the data for now, and then either hard-code the program to put the data in the right place, or have the code find the right cell; or else try to standardize the spreadsheets. That could be problematic because I'm not the only one with access to it, and if another end-user changes the sheet without my knowledge, then the code could mess everything up if it's automated.

## Job Function

I want to start with the easiest one, so I'll be populating job function tables first.

In [3]:
visitors_job_functions = pd.read_excel(file, sheet_name=[2])

In [4]:
job_function = visitors_job_functions[2].sort_values(by="Total views", ascending=False).iloc[:5].reset_index(drop=True)

In [8]:
jobs = list(job_function["Job function"])
jobs

['Marketing',
 'Program and Project Management',
 'Business Development',
 'Arts and Design',
 'Education']

In [1]:
def get_translation(word):
    dictionary = pd.read_csv(f"{DATA_DIR}/linkedin_dictionary.csv")
    loc = dictionary.loc[dictionary["English"] == word]["Japanese"].index[0]
    return dictionary.loc[dictionary["English"] == word].iloc[:,1][loc]

In [10]:
job_function["Japanese"] = [get_translation(job) for job in jobs]

In [11]:
job_function

,Job function,Total views,Japanese
0,Marketing,22,マーケティング
1,Program and Project Management,14,プログラム／プロジェクト管理
2,Business Development,13,事業開発
3,Arts and Design,11,アートとデザイン
4,Education,9,教育


## Industry

This one is easy as well, so I'll go ahead and populate it.

In [12]:
visitors_industry = pd.read_excel(file, sheet_name=[4])

In [13]:
industry = visitors_industry[4].sort_values(by="Total views", ascending=False).iloc[:6]

In [21]:
industries = list(industry["Industry"])
industries

['Building Materials',
 'Architecture & Planning',
 'Marketing and Advertising',
 'Plastics',
 'Philanthropy',
 'International Trade and Development']

In [24]:
industry["Japanese"] = [get_translation(field) for field in industries]
industry

,Industry,Total views,Japanese
2,Building Materials,23,建築材料産業
3,Architecture & Planning,21,建築と計画
6,Marketing and Advertising,19,マーケティング・広告
7,Plastics,11,プラスチック
8,Philanthropy,10,慈善活動
9,International Trade and Development,6,国際貿易と開発


## Location

This one is a bit more difficult, because I'll have to transform the dataset first if I want to get country-level data. But first, let's be sure I can grab area-level data first.

### Area-Level Data

In [4]:
area_file = f"{DATA_DIR}/jotovent-2020-10_visitors.xls"

In [5]:
visitors_location_by_area = pd.read_excel(area_file, sheet_name=[1])

FileNotFoundError: [Errno 2] No such file or directory: '/home/jayascript/Code/jayascript/data/raw/linkedin/jotovent-2020-10_visitors.xls'

In [6]:
visitors_location_by_area[1].sort_values(by="Total views", ascending=False).iloc[:5]

NameError: name 'visitors_location_by_area' is not defined

### Country-Level Data

In [94]:
visitors_location_by_country = pd.read_excel(file, sheet_name=[1])

In [95]:
locations = visitors_location_by_country[1].sort_values(by="Total views", ascending=False)

In [96]:
locations

,Location,Total views
2,Greater Seattle Area,62
5,"Kitchener, Canada Area",5
0,Greater New York City Area,4
6,"São Paulo Area, Brazil",4
1,"Portland, Oregon Area",2
3,"Kansas City, Missouri Area",2
4,"Raleigh-Durham, North Carolina Area",2
7,"Region VII - Central Visayas, Philippines",1


In [97]:
countries = list(locations["Location"])

In [98]:
countries = [country.split(",") for country in countries]

In [99]:
countries = [country[-1].strip() for country in countries]

In [100]:
final_list = []
for country in countries:
    if country[-4:] == 'Area' and "Canada" not in country:
        final_list.append("United States")
    else:
        final_list.append(country)

In [101]:
locations["Location"] = final_list

In [102]:
import numpy as np
locations.groupby("Location").agg(np.sum).sort_values(by="Total views", ascending=False)

,Total views
Location,
United States,72
Canada Area,5
Brazil,4
Philippines,1


In [38]:
locations

,Location,Total views
0,United States,21
7,United States,17
19,India,8
23,China,6
6,United States,6
21,India,5
11,United States,5
16,Brazil,4
20,India,4
8,United States,4
